In [1]:
# import the opencv library
import cv2
from pytorch_helper import PyTorchHelper

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

from torchvision.models import resnet18
import math

In [3]:
model = torch.jit.load('output/the_best.pt')

In [4]:
helper = PyTorchHelper(0,None)

In [5]:
def predict(frame):
    height, width, depth = frame.shape
    img = cv2.resize(frame,dsize=(256, 256))
    x_data = torch.tensor(np.asarray(img))
    pred = model(x_data.unsqueeze(0))[0]
    if pred[0] > 0.5:
        x_top_left = math.ceil(pred[1] * width)
        y_top_left = math.ceil(pred[2] * height)

        x_bottom_right = math.ceil(pred[3] * width)
        y_bottom_right = math.ceil(pred[4] * height)

        frame = cv2.rectangle(frame, (x_top_left, y_top_left), (x_bottom_right, y_bottom_right),
                                (0, 0, 255), 4)
    return frame    

In [6]:

  
  
# define a video capture object
vid = cv2.VideoCapture(0)
try:
    while(True):

        # Capture the video frame
        # by frame
        ret, frame = vid.read()

        # Display the resulting frame
        frame = predict(frame)
        cv2.imshow('frame', frame)

        # the 'q' button is set as the
        # quitting button you may use any
        # desired button of your choice
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:  
    # After the loop release the cap object
    vid.release()
    # Destroy all the windows
    cv2.destroyAllWindows()

RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__/torchvision/models/resnet.py", line 22, in forward
  def forward(self: __torch__.torchvision.models.resnet.ResNet,
    x: Tensor) -> Tensor:
    return (self)._forward_impl(x, )
            ~~~~~~~~~~~~~~~~~~~ <--- HERE
  def _forward_impl(self: __torch__.torchvision.models.resnet.ResNet,
    x: Tensor) -> Tensor:
  File "code/__torch__/torchvision/models/resnet.py", line 25, in _forward_impl
  def _forward_impl(self: __torch__.torchvision.models.resnet.ResNet,
    x: Tensor) -> Tensor:
    x0 = (self.conv1).forward(x, )
          ~~~~~~~~~~~~~~~~~~~ <--- HERE
    x1 = (self.bn1).forward(x0, )
    x2 = (self.relu).forward(x1, )
  File "code/__torch__/torch/nn/modules/conv.py", line 21, in forward
  def forward(self: __torch__.torch.nn.modules.conv.Conv2d,
    input: Tensor) -> Tensor:
    _0 = (self)._conv_forward(input, 